In [1]:
!pip install rank_bm25
!pip install PyPDF2
!pip install unstructured
!pip install "unstructured[pdf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=679c7c78ea721887145cbf5a83f2e49c5afa38077f8898515d7da8a7f08730a2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.

In [2]:
from rank_bm25 import BM25Okapi
from PyPDF2 import PdfReader
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title

In [6]:
from typing import List, Tuple
from rank_bm25 import BM25Okapi

def tune_bm25(corpus: List[List[str]], query: str, k1_values: List[float], b_values: List[float]) -> Tuple[float, float]:
    best_score = 0
    best_params = (0, 0)

    for k1 in k1_values:
        for b in b_values:
            bm25 = BM25Okapi(corpus, k1=k1, b=b)
            query_tokens = query.split()
            doc_scores = bm25.get_scores(query_tokens)
            average_score = sum(doc_scores) / len(doc_scores)

            # Update the best parameters if the current configuration has a higher score
            if average_score > best_score:
                best_score = average_score
                best_params = (k1, b)

    return best_params

def rank_documents(elements, query, k1_values, b_values):
    # Convert CompositeElement objects to lists of words
    corpus = [element.text.split() for element in elements]

    # Tune hyperparameters
    best_k1, best_b = tune_bm25(corpus, query, k1_values, b_values)

    # Initialize BM25 with the best hyperparameters
    bm25 = BM25Okapi(corpus, k1=best_k1, b=best_b)

    # Tokenize the query
    query_tokens = query.split()

    # Get scores
    doc_scores = bm25.get_scores(query_tokens)

    # Pair each document with its score and metadata
    doc_score_pairs = zip(elements, doc_scores)

    # Sort the documents by their scores in descending order
    ranked_docs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

    # Return the ranked documents with their scores and metadata
    return [(i+1, score, ' '.join(doc.text.split()), doc.metadata) for i, (doc, score) in enumerate(ranked_docs)]



# partition and chunk_by_title
path = "/content/drive/MyDrive/Colab Notebooks/AI for Good/MSc5yrCSSchemeandSyllabus2021.pdf"
elements = partition(path)
corpus = chunk_by_title(elements, new_after_n_chars=1500, combine_text_under_n_chars=700)




In [7]:
# Now you can use the modified functions with the corpus
query = "I love python"
k1_values = [1.2, 1.5, 1.8]
b_values = [0.75, 0.85, 0.95]
ranked_docs = rank_documents(corpus, query, k1_values, b_values)

In [18]:
# Print the ranked documents
for rank, score, text, metadata in ranked_docs:
    print(f"Rank: {rank}, Score: {score}, Text: {text}, Metadata_pageNumber: {metadata.page_number}")

Rank: 1, Score: 7.9397177697377455, Text: Course Outcomes (CO) After the completion of the course, the students will be able to: CO1: Write, test and debug simple python programs CO2: Implement python programs with conditional statements and loops CO3: Develop Python programs stepwise by defining functions and calling them CO4: Use Python lists, tuples, dictionaries for representing compound data CO5: Read and write data from/to files in Python, Metadata_pageNumber: 19
Rank: 2, Score: 5.660442375479835, Text: I with Deep Learning by IBM (Coursera) 10. Introduction to pyTorch and Machine Learning (Udemy) 11. Practical Deep Learning with pyTorch (Udemy), Metadata_pageNumber: 115
Rank: 3, Score: 4.851022444841954, Text: NTELLIGENCE ​ & D ​ Semester - I Sl. No. Course code Course Title 21-XXX-0101 Mathematics for Data Science 21-XXX-0102 Communicative English 21-XXX-0103 Object Oriented Programming 1 2 3 C/E Cr C C C 4 3 4 Lr 4 3 4 4 5 6 7 Computational Thinking for Problem Solving 21-XXX-